## Module to perform Multimodal Video and Text Sentimental Analysis
### Sections
* PSO Based Hyper-Parameter Optimization
* Applying RFE for CountVectozier
* Applyting RFE for TF-IDF 


In [11]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import spacy

In [13]:
!pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_md-2.2.5/es_core_news_md-2.2.5.tar.gz

     |████████████████████████████████| 78.4MB 1.2MB/s 
  Created wheel for es-core-news-md: filename=es_core_news_md-2.2.5-cp36-none-any.whl size=79649483 sha256=02d65b6091da1c3ead129fe2825035b37a94433b4ff0e085ec4b7618326b9131
  Stored in directory: /root/.cache/pip/wheels/b7/bb/a3/29ab5cf80c2c0a8fa0f2af8402fdace3f159e8265f0fdcbcdb
Successfully built es-core-news-md


In [0]:
import es_core_news_md
import spacy
nlp=es_core_news_md.load()

# Pre-Processing and Feature Selection




In [0]:
data = pd.read_csv('/content/text_vid.csv')
df = pd.DataFrame(data)

In [0]:
df_v2 = df.copy()
df_V = df.copy()
a = list(df_v2.columns)
a.remove('frame')
a.remove('face_id')
a.remove('timestamp')
a.remove('confidence')
a.remove('success')
a.remove('Unnamed: 0')
b = a.copy()
b.remove('Speech')
b.remove('Sentiment')

df_v2 = df_v2[a]
df_v2 = df_v2.dropna()
df_v2 = df_v2.reset_index(drop=True)

In [0]:
#Lemmatization on individual entries, stored in a list
l=[]
for t in df_v2.Speech:
  s=''
  for i in nlp(t):
    s=s+i.lemma_+' '
  l.append(s)
df_v2['Speech']=l

In [0]:
#import stopwords
from nltk.corpus import stopwords
my_stop_words=list(stopwords.words('spanish'))

In [0]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

from bs4 import BeautifulSoup
import re
def preprocess_text(text):
    #remove html tags
    tags_rem = BeautifulSoup(text, "lxml").get_text()
    #remove non alphabet characters
    letters_only = re.sub("[^a-zA-Z]", " ", tags_rem)
    #make all words lower
    words = letters_only.lower().split()
    #fetch stop words list
    stop_word = set(stopwords.words("spanish"))
    #get list with stop words removed
    final = [w for w in words if not w in stop_word]
    #lemmatime words 
    #new_final = [lemmatizer.lemmatize(w) for w in final]
    return( " ".join(final))


df_v2['Speech'] = df_v2['Speech'].apply(lambda x: preprocess_text(x))

In [0]:
#function to plot roc_auc_curve
def plot_roc_auc(model,X_test,y_test,label):
    from sklearn.metrics import roc_curve
    from sklearn.metrics import roc_auc_score
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(y_test))]
    # predict probabilities
    lr_probs = model.predict_proba(X_test)
    #print(lr_probs)
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    #print(lr_probs)
    # calculate scores
    ns_auc = roc_auc_score(y_test, ns_probs)
    lr_auc = roc_auc_score(y_test, lr_probs)
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print('Logistic: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(lr_fpr, lr_tpr, marker='.', label=label)
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()# show the plot
    plt.show()

# Hyperparameter optimization using PSO

In [0]:
#create a diff dataframe to work with Genetic Algorithms
df_V = df_v2.copy()
y = df_V.Sentiment
X = df_V.drop(['Sentiment','Speech'], axis=1)

In [0]:
!pip install hyperactive

     |████████████████████████████████| 61kB 1.6MB/s 


In [0]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from hyperactive import Hyperactive
from xgboost import XGBClassifier


def pca(X):
    X = PCA(n_components=449).fit_transform(X)

    return X


def none(X):
    return X


def model(para, X, y):
    model = RandomForestClassifier(
        n_estimators=para["n_estimators"],
        max_depth=para["max_depth"],
        min_samples_split=para["min_samples_split"],
        min_samples_leaf=para["min_samples_leaf"],
    )
    X_pca = para["decomposition"](X)
    X = np.hstack((X, X_pca))

    X = SelectKBest(f_classif, k=para["k"]).fit_transform(X, y)
    scores = cross_val_score(model, X, y, cv=3)

    return scores.mean()


search_config = {
    model: {
        "decomposition": [pca, none],
        "k": range(2, 30),
        "n_estimators": range(10, 200, 10),
        "max_depth": range(2, 12),
        "min_samples_split": range(2, 12),
        "min_samples_leaf": range(1, 11),
    }
}


opt = Hyperactive(np.ascontiguousarray(X.to_numpy(), dtype=np.float32),np.ascontiguousarray(y.to_numpy(), dtype=np.float32))
opt.search(search_config, optimizer="ParticleSwarm",n_iter=200)

No meta data found for model function
Set random start position



best para = {'decomposition': <function none at 0x7fb84177ad90>, 'k': 14, 'n_estimators': 120, 'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 6}
score     = 0.6549440715883669 



* Optimizer outputs a list of parameters, that gives the best output and refrains us from using PCA for Feature Selection

### HPO on XGBoost model

In [0]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperactive import Hyperactive



def model(para, X, y):
    xgb = XGBClassifier(
        n_estimators=para["n_estimators"],
        max_depth=para["max_depth"],
        learning_rate=para["learning_rate"],
    )
    scores = cross_val_score(xgb, X, y, cv=3)

    return scores.mean()


search_config = {
    model: {
        "n_estimators": range(10, 200, 10),
        "max_depth": range(2, 12),
        "learning_rate": [1e-3, 1e-2, 1e-1, 0.5, 1.0],
    }
}


opt = Hyperactive(np.ascontiguousarray(X_train.to_numpy(), dtype=np.float32),np.ascontiguousarray(y_train.to_numpy(), dtype=np.float32))
opt.search(search_config, optimizer="ParticleSwarm",n_iter=1000)

No meta data found for model function
Set random start position



best para = {'n_estimators': 90, 'max_depth': 2, 'learning_rate': 0.1}
score     = 0.76890756302521 



## Count Vectorizer

In [22]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk import word_tokenize
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [24]:
final = []
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier 
import warnings
warnings.filterwarnings('ignore')
#Initializing RFE model
#specify the range for feature size
for i in range(500,709,10):
  #create a new model for every feature selection
  model = DecisionTreeClassifier()
  rfe = RFE(model, i)
  #Transforming data using RFE
  X_rfe = rfe.fit_transform(X,y)  
  #Fitting the data to model
  model.fit(X_rfe,y)
  a= list(rfe.support_)
  b = []
  #get the list of retained features
  for j in range(len(a)):
    if a[j]==True:
      b.append(list(X.columns)[j])
  X2 = X[b]
  print('Debug,',len(X2.columns))
  vect=CountVectorizer()
  vect.fit(df_v2.Speech)
  X_speech=vect.transform(df_v2.Speech)
  X_df = pd.DataFrame(X_speech.toarray(), columns=vect.get_feature_names())
  X2['Sentiment'] = y
  horizontal_stack = pd.concat([X_df, X2], axis=1)
  y = horizontal_stack.Sentiment
  X_n = horizontal_stack.drop('Sentiment', axis=1)
  # Perform the train-test split
  X_train, X_test, y_train, y_test = train_test_split(X_n, y, test_size=0.2, random_state=42, stratify=y)
  xg_model = XGBClassifier()
  #Train XGB Model
  xg_model.fit(X_train,y_train)
  #Predict the response for test dataset
  y_predicted3 = xg_model.predict(X_test)
  c = (i,accuracy_score(y_test, y_predicted3))
  print(c)
  final.append(c)





Debug, 500
(500, 0.8777777777777778)
Debug, 510
(510, 0.8333333333333334)
Debug, 520
(520, 0.8333333333333334)
Debug, 530
(530, 0.8333333333333334)
Debug, 540
(540, 0.8333333333333334)
Debug, 550
(550, 0.8444444444444444)
Debug, 560
(560, 0.8666666666666667)
Debug, 570
(570, 0.8555555555555555)
Debug, 580
(580, 0.8666666666666667)
Debug, 590
(590, 0.8555555555555555)
Debug, 600
(600, 0.8555555555555555)
Debug, 610
(610, 0.8222222222222222)
Debug, 620
(620, 0.8333333333333334)
Debug, 630
(630, 0.8444444444444444)
Debug, 640
(640, 0.8666666666666667)
Debug, 650
(650, 0.8333333333333334)
Debug, 660
(660, 0.8333333333333334)
Debug, 670
(670, 0.8555555555555555)
Debug, 680
(680, 0.8666666666666667)
Debug, 690
(690, 0.8444444444444444)
Debug, 700
(700, 0.8444444444444444)


# TF-IDF

In [0]:
final = []
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix



#Initializing RFE model
for i in range(500,709,10):
  model = DecisionTreeClassifier()
  rfe = RFE(model, i)
  #Transforming data using RFE
  X_rfe = rfe.fit_transform(X,y)  
  #Fitting the data to model
  model.fit(X_rfe,y)
  a= list(rfe.support_)
  b = []
  for j in range(len(a)):
    if a[j]==True:
      b.append(list(X.columns)[j])
  X2 = X[b]
  print('Debug,',len(X2.columns))
  vect=TfidfVectorizer() #Remove stopwords
  vect.fit(df_v2.Speech)
  X_speech=vect.transform(df_v2.Speech)
  X_df = pd.DataFrame(X_speech.toarray(), columns=vect.get_feature_names())
  X2['Sentiment'] = y
  horizontal_stack = pd.concat([X_df, X2], axis=1)
  y = horizontal_stack.Sentiment
  X_n = horizontal_stack.drop('Sentiment', axis=1)
  # Perform the train-test split
  X_train, X_test, y_train, y_test = train_test_split(X_n, y, test_size=0.2, random_state=42, stratify=y)
  xg_model = XGBClassifier()
  #Train XGB Model
  xg_model.fit(X_train,y_train)
  #Predict the response for test dataset
  y_predicted3 = xg_model.predict(X_test)
  c = (i,accuracy_score(y_test, y_predicted3))
  print(c)
  final.append(c)

Debug, 500
(500, 0.8333333333333334)
Debug, 510
(510, 0.8444444444444444)
Debug, 520
(520, 0.8555555555555555)
Debug, 530
(530, 0.8444444444444444)
Debug, 540
(540, 0.8666666666666667)
Debug, 550
(550, 0.8666666666666667)
Debug, 560
(560, 0.8444444444444444)
Debug, 570
(570, 0.8555555555555555)
Debug, 580
(580, 0.8333333333333334)
Debug, 590
(590, 0.8222222222222222)
Debug, 600
(600, 0.8444444444444444)
Debug, 610
(610, 0.8333333333333334)
Debug, 620
(620, 0.8111111111111111)
Debug, 630
(630, 0.8444444444444444)
Debug, 640
(640, 0.8444444444444444)
Debug, 650
(650, 0.8555555555555555)
Debug, 660
(660, 0.8555555555555555)
Debug, 670
(670, 0.8333333333333334)
Debug, 680
(680, 0.8666666666666667)
Debug, 690
(690, 0.8666666666666667)
Debug, 700
(700, 0.8444444444444444)


# Conclusion

* Feature Reduction does increase the score.
* PCA isn't the right choice for feature reduction.

## Plotting

In [33]:
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import tools

trace1 = go.Bar(
    name='Setting Up the Baselines',
    x=['T', 'V', 'T+V'],
    y=[47.68 ,48.40 ,49.22]
)
trace2 = go.Bar(
    name='Benchmarking Multimodal Sentiment Analysis',
    x=['T', 'V', 'T+V'],
    y=[49.6,46.9 ,49.8],
)
trace3 = go.Bar(
    name='MSA of Spanish Online Videos',
    x=['T', 'V', 'T+V'],
    y=[61.04 ,64.94 ,73.68],
)
trace4 = go.Bar(
    name='MSA To Explore the Structure of Emotions',
    x=['T', 'V', 'T+V'],
    y=[50.66,73.33 ,74.66],
)
trace5 = go.Bar(
    name='Harvesting opinions from the web',
    x=['T', 'V', 'T+V'],
    y=[67.31 ,70.94 ,72.39],
)
trace6 = go.Bar(
    name='Fusing audio, visual and textual clues for SA',
    x=['T', 'V', 'T+V'],
    y=[62.11 ,78.00 ,78.80],
)
trace7 = go.Bar(
    name='Tensor Fusion Network for Multimodal Sentiment Analysis',
    x=['T', 'V', 'T+V'],
    y=[87.83 , 83.48,89.57],
)
trace8 = go.Bar(
    name='Neural Networks and Multiple Kernel learning for multimodal sentiment analysis',
    x=['T', 'V', 'T+V'],
    y=[61.60 ,76.48,77.17],
)
trace9 = go.Bar(
    name='Our Implementation',
    x=['T', 'V', 'T+V'],
    y=[84.44,84.44,88.89],
)

fig = tools.make_subplots(rows=1, cols=9,
                          shared_xaxes=True, shared_yaxes=True,
                          vertical_spacing=0.002)
fig.update_layout(legend_orientation="h")
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 1, 4)
fig.append_trace(trace5, 1, 5)
fig.append_trace(trace6, 1, 6)
fig.append_trace(trace7, 1, 7)
fig.append_trace(trace8, 1, 8)
fig.append_trace(trace9, 1, 9)

fig['layout'].update(height=650, width=800, title='Model Scores', yaxis_title="Accuracy")
fig.show()
pyo.plot(fig, filename='bar-charts-with-shared-axis.html')

'bar-charts-with-shared-axis.html'

In [0]:
import plotly.graph_objects as go

months = ['Random Forest English','Random Forest Spanish','XGBoost English','XGBoost Spanish']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=[80, 74.44, 76.66, 72.22],
    width=0.3,
    name='Text',
    marker_color='red'
))
fig.add_trace(go.Bar(
    x=months,
    y=[86.66, 83.33, 86.66, 82.22],
    name='Text + Vid',
    marker_color='blue',
    width=0.3
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45 ,title="Model Accuracy Scores",xaxis_title="Model",yaxis_title="Accuracy",)

fig.show()

In [32]:
import plotly.graph_objects as go

months = num

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=val,
    name='Text',
    marker_color='orange',
))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45 ,title="Model Accuracy Scores",xaxis_title="Feature Size",yaxis_title="Accuracy",)

fig.show()